# KNN (K-Nearest-Neighbors)

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('/Users/Zia/Google Drive/Bootcamp/Bootcamp Notes/Day 6 Recommendation Systems Notes/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [4]:
movieNumRatings2 = pd.DataFrame(movieProperties['rating']['mean'])
movieNumRatings2.head()

,mean
movie_id,
1,3.878319
2,3.206107
3,3.033333
4,3.550239
5,3.302326


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [5]:
import pandas as pd
df = pd.read_csv('/Users/Zia/Google Drive/Bootcamp/Bootcamp Notes/Day 6 Recommendation Systems Notes/ml-100k/u.item', sep='|', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
0     1682 non-null int64
1     1682 non-null object
2     1681 non-null object
3     0 non-null float64
4     1679 non-null object
5     1682 non-null int64
6     1682 non-null int64
7     1682 non-null int64
8     1682 non-null int64
9     1682 non-null int64
10    1682 non-null int64
11    1682 non-null int64
12    1682 non-null int64
13    1682 non-null int64
14    1682 non-null int64
15    1682 non-null int64
16    1682 non-null int64
17    1682 non-null int64
18    1682 non-null int64
19    1682 non-null int64
20    1682 non-null int64
21    1682 non-null int64
22    1682 non-null int64
23    1682 non-null int64
dtypes: float64(1), int64(20), object(3)
memory usage: 315.4+ KB


In [27]:
df.loc[:2,5:23].values.tolist()

[[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]

In [8]:
movieNormalizedNumRatings.head(2)

,size
movie_id,
1,0.773585
2,0.222985


In [9]:
movieDict = {}
with open(r'/Users/Zia/Google Drive/Bootcamp/Bootcamp Notes/Day 6 Recommendation Systems Notes/ml-100k/u.item') as f:
    
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:24]
        genres = map(int, genres)
        movieDict[movieID] = (name, genres, movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


In [10]:
genres

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

For example, here's the record we end up with for movie ID 1, "Toy Story":

In [11]:
movieDict

{1: ('Toy Story (1995)',
  [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  0.77358490566037741,
  3.8783185840707963),
 2: ('GoldenEye (1995)',
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  0.22298456260720412,
  3.2061068702290076),
 3: ('Four Rooms (1995)',
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  0.15265866209262435,
  3.0333333333333332),
 4: ('Get Shorty (1995)',
  [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  0.35677530017152659,
  3.5502392344497609),
 5: ('Copycat (1995)',
  [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  0.14579759862778729,
  3.3023255813953489),
 6: ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  0.042881646655231559,
  3.5769230769230771),
 7: ('Twelve Monkeys (1995)',
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  0.67066895368782165,
  3.7984693877551021),
 8: ('Babe (1995)',
  [0, 0, 0, 0,

In [12]:
movieDict[2][0], movieDict[4][0]

('GoldenEye (1995)', 'Get Shorty (1995)')

In [13]:
movieDict[2][1], movieDict[4][1]

([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [14]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    




Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

In [16]:
print movieDict[2]
print movieDict[4]


('GoldenEye (1995)', [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.35677530017152659, 3.5502392344497609)


In [17]:
ComputeDistance(movieDict[2], movieDict[4])

0.8004574042309891

Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. Sort them by distance, and print out the K nearest neighbors:

In [29]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 30
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print movieDict[neighbor][0] + " " + str(movieDict[neighbor][3])
    


Liar Liar (1997) 3.15670103093
Aladdin (1992) 3.81278538813
Willy Wonka and the Chocolate Factory (1971) 3.63190184049
Monty Python and the Holy Grail (1974) 4.0664556962
Full Monty, The (1997) 3.92698412698
George of the Jungle (1997) 2.68518518519
Beavis and Butt-head Do America (1996) 2.78846153846
Birdcage, The (1996) 3.44368600683
Home Alone (1990) 3.08759124088
Aladdin and the King of Thieves (1996) 2.84615384615
Lion King, The (1994) 3.78181818182
Jungle2Jungle (1997) 2.43939393939
Babe (1995) 3.99543378995
Wrong Trousers, The (1993) 4.46610169492
Raising Arizona (1987) 3.875
Beauty and the Beast (1991) 3.79207920792
Back to the Future (1985) 3.83428571429
101 Dalmatians (1996) 2.90825688073
Fish Called Wanda, A (1988) 3.78542510121
Pinocchio (1940) 3.67326732673
Matilda (1996) 3.21052631579
Mary Poppins (1964) 3.72471910112
In & Out (1997) 3.30434782609
Fantasia (1940) 3.77011494253
Snow White and the Seven Dwarfs (1937) 3.70930232558
Santa Clause, The (1994) 3.09756097561
Swor

While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [30]:
avgRating

104.82306167105551

In [31]:
avgRating /=float(K)
avgRating

3.4941020557018505

How does this compare to Toy Story's actual average rating?

In [20]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 0.77358490566037741,
 3.8783185840707963)

Not too bad!


## Assignment 

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

In [31]:
K = 30
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print movieDict[neighbor][0] + " " + str(movieDict[neighbor][3])
    

Liar Liar (1997) 3.15670103093
Aladdin (1992) 3.81278538813
Willy Wonka and the Chocolate Factory (1971) 3.63190184049
Monty Python and the Holy Grail (1974) 4.0664556962
Full Monty, The (1997) 3.92698412698
George of the Jungle (1997) 2.68518518519
Beavis and Butt-head Do America (1996) 2.78846153846
Birdcage, The (1996) 3.44368600683
Home Alone (1990) 3.08759124088
Aladdin and the King of Thieves (1996) 2.84615384615
Lion King, The (1994) 3.78181818182
Jungle2Jungle (1997) 2.43939393939
Babe (1995) 3.99543378995
Wrong Trousers, The (1993) 4.46610169492
Raising Arizona (1987) 3.875
Beauty and the Beast (1991) 3.79207920792
Back to the Future (1985) 3.83428571429
101 Dalmatians (1996) 2.90825688073
Fish Called Wanda, A (1988) 3.78542510121
Pinocchio (1940) 3.67326732673
Matilda (1996) 3.21052631579
Mary Poppins (1964) 3.72471910112
In & Out (1997) 3.30434782609
Fantasia (1940) 3.77011494253
Snow White and the Seven Dwarfs (1937) 3.70930232558
Santa Clause, The (1994) 3.09756097561
Swor

In [27]:
avgRating /=float(K)
avgRating

3.4941020557018505